In [109]:
from openai import OpenAI

In [111]:
gemini_api_key = "*******************************************"

In [113]:
model = OpenAI(api_key = gemini_api_key , base_url = "https://generativelanguage.googleapis.com/v1beta/openai/")

In [1]:
import gradio as gr
import google.generativeai as genai
import traceback

# ✅ Configure Gemini API
genai.configure(api_key="**********************************************")

# ✅ Use working model (you can switch to 'gemini-1.5-pro' if you prefer)
model = genai.GenerativeModel("gemini-1.5-flash")

# ✅ Prompt generator
def generate_prompt(budget, mileage, fuel_type, transmission, car_type):
    return f"""
Suggest the best car available in India based on the following requirements:
- Budget: ₹{budget}
- Mileage: {mileage} kmpl or better
- Fuel Type: {fuel_type}
- Transmission: {transmission}
- Car Type: {car_type}

Provide output in the following format:
Car Name: ...
Price: ...
Mileage: ...
Fuel Type: ...
Transmission: ...
Car Type: ...
Short Description: ...
Image URL: (only direct image URL)
"""

# ✅ Car recommendation function
def recommend_car(budget, mileage, fuel_type, transmission, car_type):
    try:
        prompt = generate_prompt(budget, mileage, fuel_type, transmission, car_type)
        print("Sending Prompt:\n", prompt)

        # Gemini API Call
        response = model.generate_content(prompt)
        result = response.text.strip()
        print("Raw Gemini Response:\n", result)

        # Extract image URL
        image_url = ""
        for line in result.splitlines():
            if line.strip().startswith("http") or "http" in line:
                image_url = line.strip()
                break

        if not image_url.startswith("http"):
            image_url = None

        text_output = result.replace(image_url, "").strip() if image_url else result

        return text_output, image_url

    except Exception as e:
        # Print traceback for debugging
        print("❌ Error Occurred:")
        traceback.print_exc()

        # Return error message in Gradio UI
        return "An error occurred while fetching car recommendation. Check console.", None

# ✅ Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("## 🚗 Car Recommendation Chatbot")

    with gr.Row():
        budget = gr.Number(label="Budget (₹)", value=1000000)
        mileage = gr.Number(label="Minimum Mileage (km/l)", value=15)

    with gr.Row():
        fuel_type = gr.Dropdown(["Petrol", "Diesel", "CNG", "Electric"], label="Fuel Type")
        transmission = gr.Dropdown(["Manual", "Automatic"], label="Transmission")

    car_type = gr.Dropdown(["Hatchback", "Sedan", "SUV", "MUV", "Compact SUV"], label="Car Type")

    submit_btn = gr.Button("Suggest Best Car 🚘")

    output_text = gr.Textbox(label="Car Recommendation", lines=10)
    output_img = gr.Image(label="Car Image")

    submit_btn.click(fn=recommend_car,
                     inputs=[budget, mileage, fuel_type, transmission, car_type],
                     outputs=[output_text, output_img])

# ✅ Run the app
demo.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [77]:
recommend_car(1000000, 15, "petrol", "automatic", "hatchback")

("Car Name: Maruti Suzuki Baleno Zeta AMT\nPrice: ₹9.15 Lakh (ex-showroom)\nMileage: 22.94 kmpl\nFuel Type: Petrol\nTransmission: Automatic (AMT)\nCar Type: Hatchback\nShort Description: The Maruti Suzuki Baleno Zeta AMT offers a good balance of features, fuel efficiency, and a comfortable ride. It's spacious for a hatchback and comes with modern features like a touchscreen infotainment system, keyless entry, and push-button start. The AMT gearbox provides a convenient driving experience.",
 'Image URL: https://imgd.aeplcdn.com/0x0/n/cw/ec/52837/baleno-exterior-right-front-three-quarter-4.jpeg?q=75')